# Phase 2: Formation Transitions

**Prerequisite**: Complete `phase1_line_formation.ipynb` first.

**Goal**: Train the NCA to transition between formations based on spatial guidance.

**Training Coverage**:
- **Formation → Formation** (25 pairs): All pairwise transitions between 5 formations
- **Seed → Formation** (5 pairs): Gaussian blob to any formation (retains Phase 1 capability)

**Formations**:
- Line (horizontal band)
- Phalanx (deep rectangular block)
- Square (hollow defensive square)
- Wedge (triangle/arrow)
- Column (vertical band)

**Key Improvements**:
- **Alpha-only loss**: Focused gradient on shape channel
- **Spatial guidance**: Target alpha broadcast as formation signal
- **Mass conservation**: Maintained during all transitions
- **Mixed pool**: 80% formations + 20% Gaussian seeds

**Success Criteria**:
- Formation→Formation: Avg MSE < 0.02, worst < 0.05
- Seed→Formation: Avg MSE < 0.02, worst < 0.05
- Mass retention > 85% average, > 70% worst case

In [1]:
# Setup
import subprocess
import sys
import os

REPO_URL = "https://github.com/JackHopkins/FormationHNCA.git"

if os.path.exists("/content"):
    REPO_DIR = "/content/FormationHNCA"
elif os.path.exists("/workspace"):
    REPO_DIR = "/workspace/FormationHNCA"
else:
    REPO_DIR = os.path.expanduser("~/FormationHNCA")

if os.path.exists(REPO_DIR):
    print(f"Pulling latest changes in {REPO_DIR}...")
    result = subprocess.run(["git", "-C", REPO_DIR, "pull"], capture_output=True, text=True)
    print(result.stdout or "Already up to date.")
else:
    print(f"Cloning repository to {REPO_DIR}...")
    subprocess.run(["git", "clone", REPO_URL, REPO_DIR], check=True)

os.chdir(REPO_DIR)

print("Installing JAX with CUDA support...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "jax[cuda12]"], check=True)

print("Installing battle-nca package...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-e", "."], check=True)

src_path = os.path.join(REPO_DIR, "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print(f"\nWorking directory: {os.getcwd()}")
print("Setup complete!")

Pulling latest changes in /workspace/FormationHNCA...
Already up to date.

Installing JAX with CUDA support...
Installing battle-nca package...

Working directory: /workspace/FormationHNCA
Setup complete!


In [2]:
import jax
import gc

gc.collect()
jax.clear_caches()

print(f"JAX version: {jax.__version__}")
print(f"JAX devices: {jax.devices()}")

if jax.devices()[0].platform == 'gpu':
    print("GPU acceleration enabled!")
else:
    print("WARNING: Running on CPU.")

JAX version: 0.8.2
JAX devices: [CudaDevice(id=0)]
GPU acceleration enabled!


In [3]:
import jax
import jax.numpy as jnp
from flax import linen as nn
from flax.training import train_state
import optax
import matplotlib.pyplot as plt
import numpy as np
from functools import partial
import time
import pickle
from pathlib import Path

from battle_nca.core import NCA, perceive
from battle_nca.core.nca import create_seed
from battle_nca.hierarchy import ChildNCA, ParentNCA, HierarchicalNCA
from battle_nca.hierarchy.child_nca import create_army_seed, CHILD_CHANNELS
from battle_nca.combat import FormationTargets, create_formation_target, rotate_formation
from battle_nca.combat.formations import FormationTypes
from battle_nca.training import NCAPool, Trainer, TrainingConfig
from battle_nca.training.optimizers import create_optimizer, normalize_gradients

print("All imports successful!")

All imports successful!


## Configuration

In [4]:
GRID_SIZE = 64
NUM_CHANNELS = 24

# RESET: Set True to start Phase 2 from Phase 1 checkpoint
# Set False to resume Phase 2 training
RESET = True

config = TrainingConfig(
    batch_size=12,
    pool_size=1024,
    min_steps=128,
    max_steps=192,
    learning_rate=2e-3,
    gradient_clip=1.0,
    damage_samples=3,
    damage_start_epoch=1000,  # Later damage since transitions are harder
    log_interval=10,
    checkpoint_interval=100
)

PHASE2_EPOCHS = 500
SEED = 42

# Rotation augmentation
USE_ROTATION_AUGMENTATION = True
ROTATION_CONTINUOUS = True

# Checkpoints
PHASE1_CHECKPOINT = Path('checkpoints/phase1_line.pkl')
PHASE2_CHECKPOINT = Path('checkpoints/phase2_transitions.pkl')
PHASE2_CHECKPOINT.parent.mkdir(exist_ok=True)

print(f"Phase 2: Formation Transitions")
print(f"Task: Formation A -> Formation B (not seed -> formation)")
print(f"Steps per transition: {config.min_steps}-{config.max_steps}")

Phase 2: Formation Transitions
Task: Formation A -> Formation B (not seed -> formation)
Steps per transition: 128-192


## Load Phase 1 Model

In [5]:
child_nca = ChildNCA(
    num_channels=NUM_CHANNELS,
    hidden_dim=128,
    fire_rate=0.5,
    use_circular_padding=True,
    use_alive_masking=False  # Disabled for Phase 2 - allows mass to grow in empty regions
)

# Seed for initialization only
seed = create_army_seed(
    height=GRID_SIZE,
    width=GRID_SIZE,
    team_color=(1.0, 0.0, 0.0),
    unit_type=0,
    formation_id=0,
    spawn_region=(GRID_SIZE//2 - 2, GRID_SIZE//2 + 2, GRID_SIZE//2 - 2, GRID_SIZE//2 + 2)
)

print(f"Seed shape: {seed.shape}")
print(f"Alive masking: DISABLED (allows transitions through empty regions)")

TypeError: ChildNCA.__init__() got an unexpected keyword argument 'use_alive_masking'

In [ ]:
key = jax.random.PRNGKey(SEED)
key, init_key = jax.random.split(key)

def count_params(params):
    return sum(p.size for p in jax.tree_util.tree_leaves(params))

# Phase 2 trains FROM SCRATCH with 6-channel signal:
# [target_alpha, flow_x, flow_y, proximity, grad_x, grad_y]
# This gives explicit "marching orders" to cells

# Number of signal channels (must match cell-16)
INIT_SIGNAL_CHANNELS = 6

# loaded_checkpoint will be used by cell-18 to restore training state
loaded_checkpoint = None

if RESET:
    print("Initializing FRESH model for Phase 2 (with 6-channel FLOW FIELD signal)...")
    
    # Create a dummy signal with 6 channels for initialization
    dummy_signal = jnp.zeros((GRID_SIZE, GRID_SIZE, INIT_SIGNAL_CHANNELS))
    
    # Initialize fresh parameters
    params = child_nca.init(init_key, seed, jax.random.PRNGKey(0), parent_signal=dummy_signal)['params']
    
    print(f"  Fresh model initialized with {count_params(params):,} parameters")
    print(f"  Signal channels: {INIT_SIGNAL_CHANNELS}")
    print(f"    [0] Target alpha (what to become)")
    print(f"    [1] Flow X (horizontal marching direction)")
    print(f"    [2] Flow Y (vertical marching direction)")
    print(f"    [3] Proximity (distance to target)")
    print(f"    [4] Target gradient X")
    print(f"    [5] Target gradient Y")
else:
    if PHASE2_CHECKPOINT.exists():
        print(f"Loading Phase 2 checkpoint from {PHASE2_CHECKPOINT}...")
        with open(PHASE2_CHECKPOINT, 'rb') as f:
            loaded_checkpoint = pickle.load(f)
        
        params = loaded_checkpoint['params']
        
        # Display checkpoint info
        epochs_done = loaded_checkpoint.get('epoch', len(loaded_checkpoint['metrics'].get('losses', [])))
        best_loss = loaded_checkpoint.get('best_loss', 'N/A')
        has_pool = 'pool' in loaded_checkpoint
        pool_shape = loaded_checkpoint['pool'].shape if has_pool else 'N/A'
        
        print(f"  Loaded Phase 2 model")
        print(f"  Epochs completed: {epochs_done}")
        print(f"  Best loss: {best_loss}")
        print(f"  Pool saved: {has_pool} (shape: {pool_shape})")
        print(f"  Metrics restored: {len(loaded_checkpoint['metrics'].get('losses', []))} loss values")
        
        if has_pool:
            print(f"\n  Training will RESUME from epoch {epochs_done + 1}")
        else:
            print(f"\n  WARNING: No pool in checkpoint, will use fresh pool")
        
        # Note: If resuming from old checkpoint with 4-channel signal,
        # model architecture will be incompatible - need fresh start
        print(f"\n  NOTE: If checkpoint used different signal channels, set RESET=True")
    else:
        print("No Phase 2 checkpoint found. Initializing fresh model...")
        dummy_signal = jnp.zeros((GRID_SIZE, GRID_SIZE, INIT_SIGNAL_CHANNELS))
        params = child_nca.init(init_key, seed, jax.random.PRNGKey(0), parent_signal=dummy_signal)['params']
        print(f"  Fresh model initialized with {count_params(params):,} parameters")

print(f"\nTotal parameters: {count_params(params):,}")

## Create Formation Targets

In [ ]:
# Create formation targets with IDENTICAL MASS via SIZE SCALING
# Instead of reducing density, we scale formation dimensions smaller
# This preserves the characteristic density of each formation type

# First, create raw formations to measure their masses
raw_targets = {
    'line': FormationTargets.line(GRID_SIZE, GRID_SIZE),
    'phalanx': FormationTargets.phalanx(GRID_SIZE, GRID_SIZE, depth=8),
    'square': FormationTargets.square(GRID_SIZE, GRID_SIZE, thickness=3),
    'wedge': FormationTargets.wedge(GRID_SIZE, GRID_SIZE),
    'column': FormationTargets.column(GRID_SIZE, GRID_SIZE, col_width=4),
}

# Compute masses before normalization
print("Formation masses (original):")
for name, target in raw_targets.items():
    mass = float(jnp.sum(target[..., 3]))
    density = float(jnp.mean(target[..., 3][target[..., 3] > 0.1]))  # Mean of non-zero
    area = int(jnp.sum(target[..., 3] > 0.1))
    print(f"  {name:10s}: mass={mass:>6.1f}, density={density:.2f}, area={area}")

# Use the SMALLEST mass as reference
masses = {name: float(jnp.sum(target[..., 3])) for name, target in raw_targets.items()}
REFERENCE_MASS = min(masses.values())
reference_formation = min(masses, key=masses.get)
print(f"\nReference: {reference_formation} with mass={REFERENCE_MASS:.1f}")


def scale_formation_to_mass(target, target_mass, grid_size):
    """Scale a formation by resizing it to achieve target mass while preserving density.
    
    The formation is scaled down (or up) spatially so that its total mass
    matches target_mass, without changing the density of filled pixels.
    
    Args:
        target: Formation target (H, W, 4)
        target_mass: Desired total mass
        grid_size: Size of the grid
    
    Returns:
        Scaled formation centered in the grid
    """
    current_mass = jnp.sum(target[..., 3])
    
    # Scale factor for dimensions (sqrt because area scales with square)
    scale = jnp.sqrt(target_mass / (current_mass + 1e-8))
    
    if scale >= 0.99:  # Already at or below target mass
        return target
    
    # Compute new dimensions
    h, w = target.shape[:2]
    new_h = int(h * scale)
    new_w = int(w * scale)
    
    # Create coordinate grids for the scaled formation
    center_y, center_x = h / 2, w / 2
    
    # Output coordinates
    y_out, x_out = jnp.meshgrid(
        jnp.arange(h, dtype=jnp.float32),
        jnp.arange(w, dtype=jnp.float32),
        indexing='ij'
    )
    
    # Map to source coordinates (inverse scaling from center)
    y_src = (y_out - center_y) / scale + center_y
    x_src = (x_out - center_x) / scale + center_x
    
    # Bilinear interpolation
    y0 = jnp.floor(y_src).astype(jnp.int32)
    x0 = jnp.floor(x_src).astype(jnp.int32)
    y1 = y0 + 1
    x1 = x0 + 1
    
    # Clamp to valid range
    y0_c = jnp.clip(y0, 0, h - 1)
    y1_c = jnp.clip(y1, 0, h - 1)
    x0_c = jnp.clip(x0, 0, w - 1)
    x1_c = jnp.clip(x1, 0, w - 1)
    
    # Interpolation weights
    wy = y_src - y0.astype(jnp.float32)
    wx = x_src - x0.astype(jnp.float32)
    
    # Out of bounds mask
    in_bounds = (
        (y_src >= 0) & (y_src < h - 1) &
        (x_src >= 0) & (x_src < w - 1)
    )
    
    def interpolate_channel(channel_data):
        v00 = channel_data[y0_c, x0_c]
        v01 = channel_data[y0_c, x1_c]
        v10 = channel_data[y1_c, x0_c]
        v11 = channel_data[y1_c, x1_c]
        
        v0 = v00 * (1 - wx) + v01 * wx
        v1 = v10 * (1 - wx) + v11 * wx
        v = v0 * (1 - wy) + v1 * wy
        
        return jnp.where(in_bounds, v, 0.0)
    
    # Apply to all channels
    scaled = jnp.stack([
        interpolate_channel(target[..., i])
        for i in range(target.shape[-1])
    ], axis=-1)
    
    return scaled


# Scale all formations to reference mass
targets = {}
for name, target in raw_targets.items():
    targets[name] = scale_formation_to_mass(target, REFERENCE_MASS, GRID_SIZE)

# Verify scaling
print("\nFormation masses AFTER size scaling:")
for name, target in targets.items():
    mass = float(jnp.sum(target[..., 3]))
    density = float(jnp.mean(target[..., 3][target[..., 3] > 0.1])) if jnp.any(target[..., 3] > 0.1) else 0
    area = int(jnp.sum(target[..., 3] > 0.1))
    print(f"  {name:10s}: mass={mass:>6.1f}, density={density:.2f}, area={area}")

formation_names = list(targets.keys())
num_formations = len(formation_names)

# Visualize base formations and rotated versions
fig, axes = plt.subplots(2, num_formations, figsize=(3*num_formations, 6))
for idx, (name, target) in enumerate(targets.items()):
    mass = float(jnp.sum(target[..., 3]))
    density = float(jnp.mean(target[..., 3][target[..., 3] > 0.1])) if jnp.any(target[..., 3] > 0.1) else 0
    
    # Base formation
    axes[0, idx].imshow(target[..., 3], cmap='gray', vmin=0, vmax=1)
    axes[0, idx].set_title(f'{name}\nmass={mass:.0f}, d={density:.2f}')
    axes[0, idx].axis('off')
    
    # Rotated 45° (worst case for clipping)
    rotated = rotate_formation(target, np.pi/4)
    rotated_mass = float(jnp.sum(rotated[..., 3]))
    axes[1, idx].imshow(rotated[..., 3], cmap='gray', vmin=0, vmax=1)
    axes[1, idx].set_title(f'rotated 45°\nmass={rotated_mass:.0f}')
    axes[1, idx].axis('off')

plt.suptitle(f'Formation Targets (SIZE-SCALED to mass={REFERENCE_MASS:.0f}, density preserved)', fontsize=14)
plt.tight_layout()
plt.show()

print(f"\nCreated {num_formations} formation targets with IDENTICAL mass via SIZE SCALING")
print(f"Density is PRESERVED - larger formations are scaled smaller to match mass")

## Create Mixed Training Pool

The pool contains both established formations AND Gaussian seeds:
- **80% formations**: For Formation A → Formation B transitions
- **20% Gaussian seeds**: For Seed → Formation transitions (Phase 1 capability)

Each training iteration will:
1. Sample a starting state (formation OR seed)
2. Pick a target formation
3. Train the NCA to transition while conserving mass

This ensures the model can:
- Transform between any pair of formations
- Form any shape from a dispersed blob (robust to damage)

In [ ]:
# === BACKGROUND ALPHA FOR GRADIENT FLOW ===
# Key insight: Empty cells (alpha=0) cannot receive useful gradients because
# there's nothing to update. By adding a small background alpha everywhere,
# we provide a gradient path for mass to "flow" into new areas.
#
# This is critical for formation transitions where the target shape
# may have mass in locations where the source has none.

BACKGROUND_ALPHA = 0.02  # Small enough to not affect visuals, large enough for gradients


def create_formation_state(target, key):
    """Create an NCA state from a formation target.
    
    Converts the target pattern into a full 24-channel state with:
    - RGB from target
    - Alpha from target WITH BACKGROUND for gradient flow
    - Other channels initialized appropriately
    """
    state = jnp.zeros((GRID_SIZE, GRID_SIZE, NUM_CHANNELS))
    
    # Copy RGB from target
    state = state.at[..., :3].set(target[..., :3])
    
    # Alpha with BACKGROUND - enables gradient flow through "empty" regions
    # The background alpha provides a path for mass to grow into new areas
    alpha_with_bg = jnp.maximum(target[..., 3], BACKGROUND_ALPHA)
    state = state.at[..., 3].set(alpha_with_bg)
    
    # Initialize health where alpha > threshold (ignoring background)
    alive = target[..., 3] > 0.1
    state = state.at[..., 4].set(jnp.where(alive, 1.0, 0.0))  # Health
    state = state.at[..., 5].set(jnp.where(alive, 1.0, 0.0))  # Morale
    
    # Small random noise in hidden channels for diversity
    noise = jax.random.uniform(key, (GRID_SIZE, GRID_SIZE, 9), minval=-0.1, maxval=0.1)
    state = state.at[..., 15:24].set(noise)
    
    return state


def create_gaussian_seed(height, width, num_channels, target_mass, key, sigma=10.0):
    """Create a Gaussian blob seed with specified total mass.
    
    Same as Phase 1 - blob centered in grid with alpha values scaled
    so total mass equals target_mass. Includes background alpha.
    """
    state = jnp.zeros((height, width, num_channels))
    
    # Create Gaussian blob centered in grid
    cy, cx = height // 2, width // 2
    y = jnp.arange(height)
    x = jnp.arange(width)
    yy, xx = jnp.meshgrid(y, x, indexing='ij')
    
    # Gaussian distribution
    dist_sq = (yy - cy) ** 2 + (xx - cx) ** 2
    gaussian = jnp.exp(-dist_sq / (2 * sigma ** 2))
    
    # Scale to match target mass (accounting for background we'll add)
    # Subtract background contribution from target
    bg_mass = height * width * BACKGROUND_ALPHA
    adjusted_target_mass = max(target_mass - bg_mass, target_mass * 0.5)  # Don't go too low
    
    current_mass = jnp.sum(gaussian)
    alpha = gaussian * (adjusted_target_mass / current_mass)
    alpha = jnp.clip(alpha, 0.0, 1.0)
    
    # Add background alpha for gradient flow
    alpha_with_bg = jnp.maximum(alpha, BACKGROUND_ALPHA)
    
    # Set RGBA channels
    state = state.at[..., 0].set(1.0)  # R (team color)
    state = state.at[..., 1].set(0.0)  # G
    state = state.at[..., 2].set(0.0)  # B
    state = state.at[..., 3].set(alpha_with_bg)  # Alpha with background
    
    # Initialize other channels where alpha > threshold (ignoring background)
    alive = alpha > 0.1
    state = state.at[..., CHILD_CHANNELS.HEALTH].set(jnp.where(alive, 1.0, 0.0))
    state = state.at[..., CHILD_CHANNELS.MORALE].set(jnp.where(alive, 0.5, 0.0))
    
    # Small noise in hidden channels
    key, subkey = jax.random.split(key)
    hidden_noise = jax.random.uniform(subkey, (height, width, 9), minval=-0.1, maxval=0.1)
    state = state.at[..., CHILD_CHANNELS.HIDDEN_START:CHILD_CHANNELS.HIDDEN_END].set(
        jnp.where(alive[..., None], hidden_noise, 0.0)
    )
    
    return state


# REFERENCE_MASS: All formations were normalized to identical mass in cell-10
# This value is used for Gaussian seeds to match formation mass
# (After normalization, any formation gives the same value)
print(f"Using normalized REFERENCE_MASS = {REFERENCE_MASS:.0f} (all formations have identical mass)")
print(f"Background alpha = {BACKGROUND_ALPHA} (enables gradient flow through empty cells)")

# Pool composition: 80% formations, 20% Gaussian seeds
SEED_FRACTION = 0.2
SEED_ID = num_formations  # Use index beyond formations to mark seeds


def create_transition_pool(pool_size, key):
    """Create a pool for transition training.
    
    Pool contains:
    - 80% established formations (for Formation A -> Formation B)
    - 20% Gaussian seeds (for Seed -> Formation, like Phase 1)
    
    This ensures the model retains seed->formation capability
    and is robust to heavily damaged/dispersed states.
    
    NOTE: All formations have IDENTICAL mass due to normalization in cell-10.
    Gaussian seeds are also scaled to this same mass.
    All states have background alpha for gradient flow.
    """
    pool_states = []
    pool_formation_ids = []  # Track which formation each state is (SEED_ID for seeds)
    
    num_seeds = int(pool_size * SEED_FRACTION)
    num_formations_samples = pool_size - num_seeds
    samples_per_formation = num_formations_samples // num_formations
    
    # Add established formations (80%)
    for formation_idx, name in enumerate(formation_names):
        target = targets[name]
        
        for i in range(samples_per_formation):
            key, subkey1, subkey2 = jax.random.split(key, 3)
            
            # Random rotation for variety
            if USE_ROTATION_AUGMENTATION:
                angle = float(jax.random.uniform(subkey1, (), minval=0, maxval=2*jnp.pi))
                rotated_target = rotate_formation(target, angle)
            else:
                rotated_target = target
            
            state = create_formation_state(rotated_target, subkey2)
            pool_states.append(state)
            pool_formation_ids.append(formation_idx)
    
    # Add Gaussian seeds (20%)
    for i in range(num_seeds):
        key, subkey = jax.random.split(key)
        # Vary sigma for different spread levels
        sigma = float(jax.random.uniform(subkey, (), minval=6.0, maxval=14.0))
        key, subkey = jax.random.split(key)
        state = create_gaussian_seed(GRID_SIZE, GRID_SIZE, NUM_CHANNELS, REFERENCE_MASS, subkey, sigma=sigma)
        pool_states.append(state)
        pool_formation_ids.append(SEED_ID)  # Mark as seed
    
    # Fill remaining slots with formations
    remaining = pool_size - len(pool_states)
    for i in range(remaining):
        key, subkey1, subkey2 = jax.random.split(key, 3)
        formation_idx = i % num_formations
        target = targets[formation_names[formation_idx]]
        
        if USE_ROTATION_AUGMENTATION:
            angle = float(jax.random.uniform(subkey1, (), minval=0, maxval=2*jnp.pi))
            rotated_target = rotate_formation(target, angle)
        else:
            rotated_target = target
        
        state = create_formation_state(rotated_target, subkey2)
        pool_states.append(state)
        pool_formation_ids.append(formation_idx)
    
    return jnp.stack(pool_states), jnp.array(pool_formation_ids)


key, pool_key = jax.random.split(key)
transition_pool, pool_formation_ids = create_transition_pool(config.pool_size, pool_key)

print(f"\nCreated transition pool: {transition_pool.shape}")
print(f"\nPool composition:")
for i, name in enumerate(formation_names):
    count = int(jnp.sum(pool_formation_ids == i))
    # Verify mass of first sample of this type
    sample_idx = int(jnp.where(pool_formation_ids == i)[0][0])
    sample_mass = float(jnp.sum(transition_pool[sample_idx, ..., 3]))
    # Mass above background
    sample_mass_active = float(jnp.sum(jnp.maximum(transition_pool[sample_idx, ..., 3] - BACKGROUND_ALPHA, 0)))
    print(f"  {name}: {count} samples, total_mass={sample_mass:.0f}, active_mass={sample_mass_active:.0f}")
seed_count = int(jnp.sum(pool_formation_ids == SEED_ID))
seed_idx = int(jnp.where(pool_formation_ids == SEED_ID)[0][0])
seed_mass = float(jnp.sum(transition_pool[seed_idx, ..., 3]))
print(f"  gaussian_seed: {seed_count} ({seed_count/config.pool_size:.0%}), mass={seed_mass:.0f}")
print(f"\nBackground mass contribution: {GRID_SIZE * GRID_SIZE * BACKGROUND_ALPHA:.0f} per state")

In [ ]:
# Visualize samples from the transition pool
fig, axes = plt.subplots(2, 5, figsize=(15, 6))

key, viz_key = jax.random.split(key)
sample_indices = jax.random.permutation(viz_key, config.pool_size)[:10]

# Extended names for display (includes seed)
display_names = formation_names + ['seed']

for i, idx in enumerate(sample_indices):
    row, col = i // 5, i % 5
    state = transition_pool[idx]
    formation_id = int(pool_formation_ids[idx])
    
    # Handle seed ID
    name = display_names[formation_id] if formation_id < len(display_names) else 'seed'
    
    axes[row, col].imshow(state[..., 3], cmap='gray', vmin=0, vmax=1)
    axes[row, col].set_title(f'{name}')
    axes[row, col].axis('off')

plt.suptitle('Transition Pool Samples (formations + seeds)', fontsize=14)
plt.tight_layout()
plt.show()

## Training Setup

In [ ]:
optimizer = create_optimizer(
    learning_rate=config.learning_rate,
    gradient_clip=config.gradient_clip
)

state = train_state.TrainState.create(
    apply_fn=child_nca.apply,
    params=params,
    tx=optimizer
)

print(f"Optimizer ready")

In [ ]:
# Training step with GRADIENT ACCUMULATION for stable loss landscape
# 1. Fixed num_steps to avoid recompilation
# 2. Vectorized checkpoint computation
# 3. Alpha-only loss for focused gradient signal
# 4. Spatial guidance via target alpha + VELOCITY FLOW FIELD (marching orders!)
# 5. Gradient accumulation over multiple micro-batches
# 6. SCHEDULED mass conservation (starts low, increases over training)
# 7. Intermediate waypoint supervision along trajectory

NUM_CHECKPOINTS = 4  # Reduced for faster training

# === SCHEDULED MASS CONSERVATION ===
MASS_CONSERVATION_WEIGHT_START = 0.05
MASS_CONSERVATION_WEIGHT_END = 0.25
MASS_CONSERVATION_WARMUP_EPOCHS = 200

VIZ_INTERVAL = 100

def get_mass_conservation_weight(epoch):
    """Get scheduled mass conservation weight for current epoch."""
    if epoch >= MASS_CONSERVATION_WARMUP_EPOCHS:
        return MASS_CONSERVATION_WEIGHT_END
    progress = epoch / MASS_CONSERVATION_WARMUP_EPOCHS
    return MASS_CONSERVATION_WEIGHT_START + progress * (MASS_CONSERVATION_WEIGHT_END - MASS_CONSERVATION_WEIGHT_START)

# FIX: Use fixed step count - INCREASED for longer trajectories
FIXED_NUM_STEPS = 128  # Reduced for faster iteration

# === GRADIENT ACCUMULATION CONFIG ===
ACCUM_STEPS = 4

# === SIGNAL CHANNELS ===
# Now using 6 channels for richer guidance:
# [target_alpha, flow_x, flow_y, distance_to_target, target_grad_x, target_grad_y]
SIGNAL_CHANNELS = 6
MODEL_SIGNAL_CHANNELS = 6  # Full signal to model


def compute_distance_field(alpha, threshold=0.1, num_iters=32):
    """Compute distance field to the nearest cell with alpha > threshold.
    
    Uses iterative relaxation (JIT-compatible with jax.lax.fori_loop).
    Returns distance normalized to [0, 1].
    """
    # Binary mask of "alive" cells
    alive = (alpha > threshold).astype(jnp.float32)
    
    h, w = alpha.shape
    max_dist = float(h + w)
    
    # Start with 0 inside, large value outside
    dist = jnp.where(alive > 0.5, 0.0, max_dist)
    
    def relaxation_step(_, dist):
        # Pad with large value
        padded = jnp.pad(dist, ((1, 1), (1, 1)), mode='constant', constant_values=max_dist)
        
        # Get minimum of neighbors + 1
        neighbors = jnp.stack([
            padded[:-2, 1:-1],  # up
            padded[2:, 1:-1],   # down
            padded[1:-1, :-2],  # left
            padded[1:-1, 2:],   # right
        ], axis=-1)
        min_neighbor = jnp.min(neighbors, axis=-1) + 1
        
        # Update distance: keep original if alive, else min of current and neighbor+1
        return jnp.where(alive > 0.5, 0.0, jnp.minimum(dist, min_neighbor))
    
    # JIT-compatible loop
    dist = jax.lax.fori_loop(0, num_iters, relaxation_step, dist)
    
    # Normalize to [0, 1]
    return dist / (max_dist + 1e-6)


def compute_flow_field(source_alpha, target_alpha, threshold=0.1):
    """Compute velocity flow field: direction each cell should move to reach target.
    
    This gives explicit "marching orders" to cells:
    - Cells outside target: move toward nearest target cell
    - Cells inside target: stay put (small/zero flow)
    
    Returns:
        flow_x: Horizontal velocity component (normalized)
        flow_y: Vertical velocity component (normalized)
        distance: Distance to target (normalized)
    """
    h, w = target_alpha.shape
    
    # Compute distance field to target
    target_dist = compute_distance_field(target_alpha, threshold)
    
    # Compute gradient of distance field = flow direction toward target
    # Use central differences
    # Pad with edge values for boundary handling
    dist_padded = jnp.pad(target_dist, ((1, 1), (1, 1)), mode='edge')
    
    # Gradient (note: negative because we want to move TOWARD low distance)
    flow_x = -(dist_padded[1:-1, 2:] - dist_padded[1:-1, :-2]) / 2.0
    flow_y = -(dist_padded[2:, 1:-1] - dist_padded[:-2, 1:-1]) / 2.0
    
    # Normalize flow vectors
    flow_mag = jnp.sqrt(flow_x**2 + flow_y**2 + 1e-8)
    flow_x = flow_x / (flow_mag + 1e-8)
    flow_y = flow_y / (flow_mag + 1e-8)
    
    # Reduce flow magnitude inside target (cells already at destination)
    inside_target = target_alpha > threshold
    flow_x = jnp.where(inside_target, flow_x * 0.1, flow_x)
    flow_y = jnp.where(inside_target, flow_y * 0.1, flow_y)
    
    return flow_x, flow_y, target_dist


def compute_target_gradients(target_alpha):
    """Compute Sobel gradients of target alpha for orientation information."""
    sobel_x = jnp.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=jnp.float32)
    sobel_y = sobel_x.T
    
    target_padded = jnp.pad(target_alpha, ((1, 1), (1, 1)), mode='wrap')
    target_4d = target_padded[None, :, :, None]
    
    grad_x = jax.lax.conv_general_dilated(
        target_4d, sobel_x[:, :, None, None], (1, 1), 'VALID',
        dimension_numbers=('NHWC', 'HWIO', 'NHWC')
    )[0, :, :, 0]
    
    grad_y = jax.lax.conv_general_dilated(
        target_4d, sobel_y[:, :, None, None], (1, 1), 'VALID',
        dimension_numbers=('NHWC', 'HWIO', 'NHWC')
    )[0, :, :, 0]
    
    # Normalize
    grad_mag = jnp.sqrt(grad_x**2 + grad_y**2 + 1e-8)
    max_mag = jnp.max(grad_mag)
    grad_x = grad_x / (max_mag + 1e-8)
    grad_y = grad_y / (max_mag + 1e-8)
    
    return grad_x, grad_y


def create_formation_signal_with_flow(batch_size, height, width, target_alpha):
    """Create goal-conditioning signal with VELOCITY FLOW FIELD.
    
    Each cell receives 6 channels of guidance:
    - Channel 0: Target alpha (what density should I become?)
    - Channel 1: Flow X (which direction should I move horizontally?)
    - Channel 2: Flow Y (which direction should I move vertically?)
    - Channel 3: Distance to target (how far am I from target?)
    - Channel 4: Target gradient X (boundary orientation)
    - Channel 5: Target gradient Y (boundary orientation)
    
    The flow field gives explicit "marching orders" - telling each cell
    WHERE to move, not just what the final state looks like. This is like
    giving soldiers actual movement commands during drill.
    """
    # Compute flow field (direction toward target)
    flow_x, flow_y, distance = compute_flow_field(
        jnp.ones_like(target_alpha) * 0.5,  # Dummy source - flow computed from target
        target_alpha
    )
    
    # Compute target gradients (boundary orientation)
    grad_x, grad_y = compute_target_gradients(target_alpha)
    
    # Build signal tensor
    signal = jnp.zeros((batch_size, height, width, SIGNAL_CHANNELS))
    signal = signal.at[..., 0].set(target_alpha)   # Target density
    signal = signal.at[..., 1].set(flow_x)         # Flow direction X
    signal = signal.at[..., 2].set(flow_y)         # Flow direction Y
    signal = signal.at[..., 3].set(1.0 - distance) # Proximity to target (inverted distance)
    signal = signal.at[..., 4].set(grad_x)         # Target gradient X
    signal = signal.at[..., 5].set(grad_y)         # Target gradient Y
    
    return signal


def create_interpolated_target(source_alpha, target_alpha, t):
    """Create an interpolated target between source and target at time t.
    
    This provides intermediate waypoints for the trajectory, giving the model
    a "trail of breadcrumbs" to follow rather than just endpoint supervision.
    
    Args:
        source_alpha: Starting formation alpha
        target_alpha: Final formation alpha  
        t: Interpolation factor [0, 1], where 0=source, 1=target
        
    Returns:
        Interpolated alpha field
    """
    # Simple linear interpolation of alpha values
    # This creates intermediate "ghost" formations
    return (1.0 - t) * source_alpha + t * target_alpha


def make_compute_grads_with_waypoints(mass_weight):
    """Create gradient computation with INTERMEDIATE WAYPOINT SUPERVISION.
    
    Instead of only supervising the final state, we supervise the trajectory
    at multiple intermediate points, creating a smoother loss landscape.
    """
    @jax.jit
    def compute_grads(params, batch, target, source_alpha_batch, key, formation_signal, initial_mass):
        """Compute gradients with waypoint supervision along trajectory."""
        num_steps = FIXED_NUM_STEPS
        
        def loss_fn(params):
            keys = jax.random.split(key, num_steps)
            
            checkpoint_interval = num_steps // NUM_CHECKPOINTS
            
            def step(carry, inputs):
                step_idx, subkey = inputs
                state_in = carry.at[..., CHILD_CHANNELS.PARENT_SIGNAL_START:CHILD_CHANNELS.PARENT_SIGNAL_END].set(
                    formation_signal[..., :2]
                )
                new_state = child_nca.apply(
                    {'params': params}, state_in, subkey, parent_signal=formation_signal[..., :MODEL_SIGNAL_CHANNELS]
                )
                return new_state, new_state
            
            step_inputs = (jnp.arange(num_steps), keys)
            final, all_states = jax.lax.scan(step, batch, step_inputs)
            
            # === CHECKPOINT EXTRACTION ===
            checkpoint_indices = jnp.array([
                (i + 1) * checkpoint_interval - 1 for i in range(NUM_CHECKPOINTS)
            ])
            checkpoint_indices = jnp.minimum(checkpoint_indices, num_steps - 1)
            checkpoint_states = all_states[checkpoint_indices]
            
            # === WAYPOINT SUPERVISION ===
            # Create interpolated targets for each checkpoint
            # Checkpoint 0 (early): closer to source
            # Checkpoint N (late): closer to target
            checkpoint_ts = jnp.array([(i + 1) / NUM_CHECKPOINTS for i in range(NUM_CHECKPOINTS)])
            
            # Compute loss against interpolated waypoints
            checkpoint_losses = []
            for i in range(NUM_CHECKPOINTS):
                t = checkpoint_ts[i]
                
                # For later checkpoints, use target directly
                # For early checkpoints, use interpolated target
                if i >= NUM_CHECKPOINTS - 2:
                    # Last 2 checkpoints: match target exactly
                    waypoint_target = target[..., 3:4]
                else:
                    # Earlier checkpoints: interpolated target
                    # This gives partial credit for being "on the way"
                    waypoint_target = target[..., 3:4]  # Still use target but with softer weight
                
                checkpoint_error = (checkpoint_states[i, ..., 3:4] - waypoint_target) ** 2
                checkpoint_losses.append(jnp.mean(checkpoint_error))
            
            checkpoint_mses = jnp.array(checkpoint_losses)
            
            # Progressive weighting: early checkpoints matter less
            checkpoint_weights = jnp.array([0.05, 0.05, 0.1, 0.1, 0.15, 0.15, 0.2, 0.2])
            formation_loss = jnp.sum(checkpoint_weights * checkpoint_mses)
            
            # === MASS CONSERVATION ===
            checkpoint_masses = jnp.sum(checkpoint_states[..., 3], axis=(2, 3))
            relative_errors = jnp.abs(checkpoint_masses - initial_mass[None, :]) / (initial_mass[None, :] + 1e-6)
            mass_conservation_loss = jnp.mean(relative_errors)
            
            total_loss = formation_loss + mass_weight * mass_conservation_loss
            return total_loss, (final, formation_loss, mass_conservation_loss)
        
        (loss, (outputs, form_loss, mass_loss)), grads = jax.value_and_grad(loss_fn, has_aux=True)(params)
        return grads, loss, outputs, form_loss, mass_loss
    
    return compute_grads


# Pre-compile gradient functions for different mass weights
MASS_WEIGHT_LEVELS = [0.05, 0.10, 0.15, 0.20, 0.25]
compute_grads_funcs = {w: make_compute_grads_with_waypoints(w) for w in MASS_WEIGHT_LEVELS}

def get_compute_grads_for_epoch(epoch):
    """Get the appropriate compute_grads function for current epoch."""
    weight = get_mass_conservation_weight(epoch)
    closest_weight = min(MASS_WEIGHT_LEVELS, key=lambda w: abs(w - weight))
    return compute_grads_funcs[closest_weight], closest_weight


@jax.jit
def apply_accumulated_grads(state, accumulated_grads, accum_steps):
    """Apply accumulated gradients after averaging."""
    avg_grads = jax.tree_util.tree_map(lambda g: g / accum_steps, accumulated_grads)
    avg_grads = normalize_gradients(avg_grads)
    state = state.apply_gradients(grads=avg_grads)
    return state


def accumulate_grads(grads1, grads2):
    """Add two gradient pytrees together."""
    return jax.tree_util.tree_map(lambda g1, g2: g1 + g2, grads1, grads2)


def zero_grads_like(params):
    """Create a zero gradient pytree with same structure as params."""
    return jax.tree_util.tree_map(jnp.zeros_like, params)


# === VISUALIZE THE NEW SIGNAL ===
print("Visualizing VELOCITY FLOW FIELD signal...")
fig, axes = plt.subplots(3, 5, figsize=(15, 9))

for idx, (name, target) in enumerate(targets.items()):
    test_signal = create_formation_signal_with_flow(1, GRID_SIZE, GRID_SIZE, target[..., 3])
    
    # Row 0: Target alpha
    axes[0, idx].imshow(test_signal[0, ..., 0], cmap='gray', vmin=0, vmax=1)
    axes[0, idx].set_title(f'{name}\n(target)')
    axes[0, idx].axis('off')
    
    # Row 1: Flow field as quiver plot
    flow_x = np.array(test_signal[0, ..., 1])
    flow_y = np.array(test_signal[0, ..., 2])
    
    # Subsample for cleaner visualization
    step = 4
    Y, X = np.mgrid[0:GRID_SIZE:step, 0:GRID_SIZE:step]
    U = flow_x[::step, ::step]
    V = flow_y[::step, ::step]
    
    axes[1, idx].imshow(test_signal[0, ..., 0], cmap='gray', vmin=0, vmax=1, alpha=0.3)
    axes[1, idx].quiver(X, Y, U, -V, color='red', scale=20, width=0.01)  # -V because imshow flips Y
    axes[1, idx].set_title('flow field\n(marching orders)')
    axes[1, idx].axis('off')
    
    # Row 2: Distance/proximity field
    axes[2, idx].imshow(test_signal[0, ..., 3], cmap='hot', vmin=0, vmax=1)
    axes[2, idx].set_title('proximity\n(1=at target)')
    axes[2, idx].axis('off')

plt.suptitle('Formation Signal with VELOCITY FLOW FIELD\n(Arrows show "where to move")', fontsize=14)
plt.tight_layout()
plt.show()

print("\nTraining configuration:")
print(f"  - Steps per iteration: {FIXED_NUM_STEPS} (increased for longer trajectories)")
print(f"  - Checkpoints: {NUM_CHECKPOINTS} (more granular supervision)")
print(f"  - Signal channels: {SIGNAL_CHANNELS}")
print(f"    [0] Target alpha")
print(f"    [1] Flow X (horizontal marching direction)")
print(f"    [2] Flow Y (vertical marching direction)")
print(f"    [3] Proximity to target (1=at target, 0=far)")
print(f"    [4] Target gradient X (boundary orientation)")
print(f"    [5] Target gradient Y (boundary orientation)")
print(f"  - Mass conservation: {MASS_CONSERVATION_WEIGHT_START} -> {MASS_CONSERVATION_WEIGHT_END}")

## Phase 2 Training: Formation Transitions

In [ ]:
# === STRATIFIED SAMPLING HELPER ===
# Ensures each batch has balanced representation from all source formation types

def get_pool_indices_by_type(pool_ids, num_types):
    """Get indices for each formation type in the pool.
    
    Returns:
        List of arrays, one per formation type (including seeds)
    """
    indices_by_type = []
    for type_id in range(num_types):
        indices = jnp.where(pool_ids == type_id)[0]
        indices_by_type.append(indices)
    return indices_by_type


def sample_stratified_batch(key, pool_indices_by_type, batch_size):
    """Sample a batch with balanced representation from available source types.
    
    Args:
        key: PRNG key
        pool_indices_by_type: List of index arrays, one per type
        batch_size: Total batch size
        
    Returns:
        Array of pool indices for the batch
    """
    # Filter to only types that have samples available
    available_types = [(i, indices) for i, indices in enumerate(pool_indices_by_type) if len(indices) > 0]
    
    if len(available_types) == 0:
        # Fallback: sample randomly from entire pool
        key, subkey = jax.random.split(key)
        total_pool_size = sum(len(indices) for indices in pool_indices_by_type)
        if total_pool_size == 0:
            raise ValueError("Pool is empty!")
        all_indices = jnp.concatenate([indices for indices in pool_indices_by_type if len(indices) > 0])
        return jax.random.choice(subkey, all_indices, shape=(batch_size,), replace=True)
    
    num_available_types = len(available_types)
    samples_per_type = batch_size // num_available_types
    remainder = batch_size % num_available_types
    
    batch_indices = []
    
    for idx, (type_id, type_indices) in enumerate(available_types):
        key, subkey = jax.random.split(key)
        
        # Number of samples for this type
        n_samples = samples_per_type
        if idx < remainder:
            n_samples += 1  # Distribute remainder across first few types
        
        # Sample from this type's indices
        if len(type_indices) >= n_samples:
            perm = jax.random.permutation(subkey, len(type_indices))[:n_samples]
            selected = type_indices[perm]
        else:
            # If not enough samples, sample with replacement
            selected = jax.random.choice(subkey, type_indices, shape=(n_samples,), replace=True)
        
        batch_indices.append(selected)
    
    return jnp.concatenate(batch_indices)


def run_trajectory_for_viz(start_state, params, key, target_alpha, num_steps=512):
    """Run a trajectory for visualization (not training).
    
    Returns states at regular intervals for animation.
    Uses the FLOW FIELD signal for guidance.
    """
    signal = create_formation_signal_with_flow(1, GRID_SIZE, GRID_SIZE, target_alpha)
    signal_single = signal[0]
    
    # Collect frames at regular intervals
    frame_interval = max(1, num_steps // 100)  # ~100 frames max
    frames = [np.array(start_state[..., 3])]
    
    state_curr = start_state
    for i in range(num_steps):
        key, subkey = jax.random.split(key)
        state_curr = state_curr.at[..., CHILD_CHANNELS.PARENT_SIGNAL_START:CHILD_CHANNELS.PARENT_SIGNAL_END].set(
            signal_single[..., :2]
        )
        state_curr = child_nca.apply(
            {'params': params}, state_curr, subkey, parent_signal=signal_single[..., :MODEL_SIGNAL_CHANNELS]
        )
        
        if (i + 1) % frame_interval == 0:
            frames.append(np.array(state_curr[..., 3]))
    
    return frames


def create_trajectory_animation(frames, target_alpha, source_name, target_name, epoch):
    """Create and display an animation of the trajectory."""
    from matplotlib.animation import FuncAnimation
    from IPython.display import HTML, display
    
    fig, axes = plt.subplots(1, 3, figsize=(12, 4))
    
    # Left: Animation of trajectory
    im = axes[0].imshow(frames[0], cmap='gray', vmin=0, vmax=1)
    axes[0].set_title(f'Step 0/{len(frames)-1}')
    axes[0].axis('off')
    
    # Middle: Target formation
    axes[1].imshow(target_alpha, cmap='gray', vmin=0, vmax=1)
    axes[1].set_title(f'Target: {target_name}')
    axes[1].axis('off')
    
    # Right: Mass over time
    masses = [np.sum(f) for f in frames]
    line, = axes[2].plot([], [], 'b-', linewidth=2)
    axes[2].axhline(y=np.sum(target_alpha), color='r', linestyle='--', label='Target mass')
    axes[2].set_xlim(0, len(frames)-1)
    axes[2].set_ylim(0, max(masses) * 1.2)
    axes[2].set_xlabel('Frame')
    axes[2].set_ylabel('Mass')
    axes[2].set_title('Mass Conservation')
    axes[2].legend()
    
    plt.suptitle(f'Epoch {epoch}: {source_name} → {target_name} ({len(frames)} frames)', fontsize=14)
    plt.tight_layout()
    
    def init():
        im.set_data(frames[0])
        line.set_data([], [])
        return im, line
    
    def animate(i):
        im.set_data(frames[i])
        axes[0].set_title(f'Frame {i}/{len(frames)-1}')
        line.set_data(range(i+1), masses[:i+1])
        return im, line
    
    anim = FuncAnimation(fig, animate, init_func=init, frames=len(frames), 
                         interval=50, blit=True)
    
    # Display as HTML5 video (works in Jupyter/Colab)
    try:
        display(HTML(anim.to_jshtml()))
    except Exception as e:
        # Fallback: show first, middle, and last frames
        print(f"Animation display failed: {e}")
        print("Showing key frames instead...")
        plt.close(fig)
        
        fig2, axes2 = plt.subplots(1, 5, figsize=(15, 3))
        key_frames = [0, len(frames)//4, len(frames)//2, 3*len(frames)//4, len(frames)-1]
        for idx, frame_idx in enumerate(key_frames):
            axes2[idx].imshow(frames[frame_idx], cmap='gray', vmin=0, vmax=1)
            axes2[idx].set_title(f'Frame {frame_idx}')
            axes2[idx].axis('off')
        plt.suptitle(f'Epoch {epoch}: {source_name} → {target_name}', fontsize=14)
        plt.tight_layout()
        plt.show()
    
    plt.close(fig)


# === PERIODIC CHECKPOINTING ===
def save_training_checkpoint(checkpoint_path, params, pool, pool_ids, epoch, best_loss, best_params,
                             losses, form_losses, mass_losses, transition_history):
    """Save a complete training checkpoint for resumption."""
    checkpoint = {
        'params': params,
        'best_params': best_params,
        'pool': np.array(pool),
        'pool_ids': np.array(pool_ids),
        'epoch': epoch,
        'best_loss': best_loss,
        'config': {
            'grid_size': GRID_SIZE,
            'num_channels': NUM_CHANNELS,
            'hidden_dim': 128,
            'goal_conditioned': True,
            'phase': 2,
            'task': 'transitions',
            'num_formations': num_formations,
            'formation_names': formation_names,
            'signal_channels': SIGNAL_CHANNELS,
        },
        'metrics': {
            'losses': losses,
            'form_losses': form_losses,
            'mass_losses': mass_losses,
            'transition_history': transition_history,
        }
    }
    
    with open(checkpoint_path, 'wb') as f:
        pickle.dump(checkpoint, f)
    
    print(f"  [Checkpoint saved: epoch {epoch}, best_loss={best_loss:.6f}]")


# Pre-compute pool indices by type for efficient stratified sampling
NUM_SOURCE_TYPES = num_formations + 1  # 5 formations + 1 seed type
pool_indices_by_type = get_pool_indices_by_type(pool_formation_ids, NUM_SOURCE_TYPES)

print("="*60)
print("PHASE 2: Formation Transitions with VELOCITY FLOW FIELD")
print("="*60)
print(f"Task: Start from Formation A (or seed), transition to Formation B")
print(f"Formations: {formation_names}")
print(f"Pool: {100*(1-SEED_FRACTION):.0f}% formations, {100*SEED_FRACTION:.0f}% Gaussian seeds")
print(f"Fixed steps per iteration: {FIXED_NUM_STEPS}")
print(f"Checkpoints: {NUM_CHECKPOINTS} (more granular supervision)")
print(f"Gradient accumulation: {ACCUM_STEPS} micro-batches (effective batch size = {config.batch_size * ACCUM_STEPS})")
print(f"STRATIFIED: Each batch has ~{config.batch_size // NUM_SOURCE_TYPES} samples per source type")
print(f"Signal: {SIGNAL_CHANNELS}-channel FLOW FIELD (marching orders!)")
print(f"Visualization: Every {VIZ_INTERVAL} epochs")
print(f"Checkpointing: Every {config.checkpoint_interval} epochs to {PHASE2_CHECKPOINT}")
print(f"Mass conservation: SCHEDULED {MASS_CONSERVATION_WEIGHT_START} -> {MASS_CONSERVATION_WEIGHT_END} over {MASS_CONSERVATION_WARMUP_EPOCHS} epochs")
print()

PATIENCE = 200
MIN_DELTA = 1e-6
VIZ_INTERVAL = 100  # Visualization with animation

# Initialize or restore training state
if not RESET and 'loaded_checkpoint' in dir() and loaded_checkpoint is not None:
    # Restore from checkpoint
    start_epoch = loaded_checkpoint.get('epoch', 0) + 1
    losses = loaded_checkpoint['metrics'].get('losses', [])
    form_losses = loaded_checkpoint['metrics'].get('form_losses', [])
    mass_losses = loaded_checkpoint['metrics'].get('mass_losses', [])
    transition_history = loaded_checkpoint['metrics'].get('transition_history', [])
    best_loss = loaded_checkpoint.get('best_loss', float('inf'))
    best_params = loaded_checkpoint.get('best_params', state.params)
    
    # Restore pool if available
    if 'pool' in loaded_checkpoint:
        current_pool = jnp.array(loaded_checkpoint['pool'])
        current_pool_ids = jnp.array(loaded_checkpoint['pool_ids'])
        print(f"Restored pool from checkpoint (shape: {current_pool.shape})")
    else:
        current_pool = transition_pool.copy()
        current_pool_ids = pool_formation_ids.copy()
        print("Pool not in checkpoint, using fresh pool")
    
    epochs_without_improvement = 0
    print(f"Resuming training from epoch {start_epoch}")
    print(f"  Previous best loss: {best_loss:.6f}")
    print(f"  Metrics restored: {len(losses)} loss values")
else:
    start_epoch = 0
    losses = []
    form_losses = []
    mass_losses = []
    transition_history = []
    best_loss = float('inf')
    best_params = state.params
    epochs_without_improvement = 0
    current_pool = transition_pool.copy()
    current_pool_ids = pool_formation_ids.copy()
    print("Starting fresh training from epoch 0")

times = []

# Track pool indices by type (will update as pool evolves)
current_pool_indices_by_type = get_pool_indices_by_type(current_pool_ids, NUM_SOURCE_TYPES)

# Extended formation names to include seed
extended_names = formation_names + ['seed']

print("Compiling JIT (first epoch will be slow)...")

for epoch in range(start_epoch, PHASE2_EPOCHS):
    start_time = time.time()
    key, epoch_key = jax.random.split(key)
    
    # Pick a TARGET formation for this epoch (same target for all micro-batches)
    epoch_key, target_key, angle_key = jax.random.split(epoch_key, 3)
    target_idx = int(jax.random.randint(target_key, (), 0, num_formations))
    target = targets[formation_names[target_idx]]
    
    # Rotation augmentation for target
    if USE_ROTATION_AUGMENTATION:
        if ROTATION_CONTINUOUS:
            angle = float(jax.random.uniform(angle_key, (), minval=0, maxval=2 * jnp.pi))
        else:
            angle = float(jax.random.randint(angle_key, (), 0, 4)) * (jnp.pi / 2)
        target = rotate_formation(target, angle)
    else:
        angle = 0.0

    # === SCHEDULED MASS CONSERVATION ===
    compute_grads_fn, current_mass_weight = get_compute_grads_for_epoch(epoch)
    
    # === GRADIENT ACCUMULATION with STRATIFIED SAMPLING ===
    accumulated_grads = zero_grads_like(state.params)
    total_loss = 0.0
    total_form_loss = 0.0
    total_mass_loss = 0.0
    all_outputs = []
    all_indices = []
    all_source_formations = []
    batch_mass_before_sum = 0.0
    batch_mass_after_sum = 0.0
    
    for accum_idx in range(ACCUM_STEPS):
        epoch_key, sample_key, step_key = jax.random.split(epoch_key, 3)
        
        # === STRATIFIED SAMPLING ===
        batch_indices = sample_stratified_batch(
            sample_key, current_pool_indices_by_type, config.batch_size
        )
        
        batch = current_pool[batch_indices]
        batch_source_formations = current_pool_ids[batch_indices]
        
        # Compute initial mass for mass conservation
        initial_mass = jnp.sum(batch[..., 3], axis=(1, 2))
        batch_mass_before_sum += float(jnp.mean(initial_mass))
        
        # Create FLOW FIELD signal (6 channels with marching orders)
        formation_signal = create_formation_signal_with_flow(
            config.batch_size, GRID_SIZE, GRID_SIZE, target[..., 3]
        )
        
        # Source alpha for waypoint supervision
        source_alpha_batch = batch[..., 3:4]
        
        # Compute gradients for this micro-batch
        grads, loss, outputs, form_loss, mass_loss = compute_grads_fn(
            state.params, batch, target, source_alpha_batch, step_key, formation_signal, initial_mass
        )
        
        # Accumulate gradients
        accumulated_grads = accumulate_grads(accumulated_grads, grads)
        
        # Track losses
        total_loss += float(loss)
        total_form_loss += float(form_loss)
        total_mass_loss += float(mass_loss)
        
        # Track outputs for pool update
        all_outputs.append(outputs)
        all_indices.append(batch_indices)
        all_source_formations.append(batch_source_formations)
        batch_mass_after_sum += float(jnp.mean(jnp.sum(outputs[..., 3], axis=(1, 2))))
    
    # Average the metrics
    avg_loss = total_loss / ACCUM_STEPS
    avg_form_loss = total_form_loss / ACCUM_STEPS
    avg_mass_loss = total_mass_loss / ACCUM_STEPS
    avg_mass_before = batch_mass_before_sum / ACCUM_STEPS
    avg_mass_after = batch_mass_after_sum / ACCUM_STEPS
    mass_retention = avg_mass_after / (avg_mass_before + 1e-6)
    
    # Apply accumulated gradients
    state = apply_accumulated_grads(state, accumulated_grads, ACCUM_STEPS)
    
    # === PERIODIC CHECKPOINT ===
    if epoch > 0 and epoch % config.checkpoint_interval == 0:
        save_training_checkpoint(
            PHASE2_CHECKPOINT, state.params, current_pool, current_pool_ids,
            epoch, best_loss, best_params, losses, form_losses, mass_losses, transition_history
        )
    
    # === VISUALIZATION ===
    if epoch % VIZ_INTERVAL == 0 and epoch > 0:
        print(f"\n{'='*60}")
        print(f"Visualization at Epoch {epoch}")
        print(f"{'='*60}")
        
        fig, axes = plt.subplots(1, 3, figsize=(14, 4))
        
        start_alpha = np.array(all_outputs[0][0, ..., 3])
        source_id = int(all_source_formations[0][0])
        source_name = extended_names[source_id] if source_id < len(extended_names) else 'seed'
        axes[0].imshow(start_alpha, cmap='gray', vmin=0, vmax=1)
        axes[0].set_title(f'Output sample\n(mass: {np.sum(start_alpha):.0f})')
        axes[0].axis('off')
        
        axes[1].imshow(np.array(target[..., 3]), cmap='gray', vmin=0, vmax=1)
        axes[1].set_title(f'Target: {formation_names[target_idx]}\n(angle: {np.degrees(angle):.0f}°)')
        axes[1].axis('off')
        
        pool_masses = [float(jnp.sum(current_pool[i, ..., 3])) for i in range(min(64, config.pool_size))]
        axes[2].hist(pool_masses, bins=20, color='steelblue', edgecolor='black')
        axes[2].set_xlabel('Mass')
        axes[2].set_title('Pool Health')
        
        plt.suptitle(f'Epoch {epoch} | Best Loss: {best_loss:.6f}')
        plt.tight_layout()
        plt.show()
        
        # Trajectory animation
        key, viz_key, state_key = jax.random.split(key, 3)
        viz_source_idx = int(jax.random.randint(viz_key, (), 0, num_formations))
        viz_source_name = formation_names[viz_source_idx]
        viz_target_name = formation_names[target_idx]
        viz_start_state = create_formation_state(targets[viz_source_name], state_key)
        
        print(f"\nGenerating trajectory animation: {viz_source_name} → {viz_target_name}...")
        
        key, traj_key = jax.random.split(key)
        frames = run_trajectory_for_viz(viz_start_state, state.params, traj_key, target[..., 3], num_steps=512)
        create_trajectory_animation(frames, np.array(target[..., 3]), viz_source_name, viz_target_name, epoch)
        
        print(f"{'='*60}\n")
    
    # Check for NaN/Inf
    if np.isnan(avg_loss) or np.isinf(avg_loss):
        print(f"\nEarly stopping at epoch {epoch}: loss is {avg_loss}")
        state = state.replace(params=best_params)
        break
    
    # Track best model
    if avg_loss < best_loss - MIN_DELTA:
        best_loss = avg_loss
        best_params = state.params
        epochs_without_improvement = 0
    else:
        epochs_without_improvement += 1
    
    if epochs_without_improvement >= PATIENCE:
        print(f"\nEarly stopping at epoch {epoch}: no improvement for {PATIENCE} epochs")
        state = state.replace(params=best_params)
        break
    
    # Pool update
    for outputs, batch_indices, batch_source_formations in zip(all_outputs, all_indices, all_source_formations):
        current_pool = current_pool.at[batch_indices].set(outputs)
        current_pool_ids = current_pool_ids.at[batch_indices].set(target_idx)
    
    current_pool_indices_by_type = get_pool_indices_by_type(current_pool_ids, NUM_SOURCE_TYPES)
    
    elapsed = time.time() - start_time
    losses.append(avg_loss)
    form_losses.append(avg_form_loss)
    mass_losses.append(avg_mass_loss)
    times.append(elapsed)
    
    # Track transition history
    source_counts = {}
    for src_formations in all_source_formations:
        for src_id in np.array(src_formations):
            src_name = extended_names[src_id] if src_id < len(extended_names) else 'seed'
            source_counts[src_name] = source_counts.get(src_name, 0) + 1
    
    source_formation = int(all_source_formations[0][0])
    transition_history.append((source_formation, target_idx))
    
    if epoch % config.log_interval == 0:
        src_summary = ", ".join([f"{k}:{v}" for k, v in sorted(source_counts.items())])
        tgt = formation_names[target_idx]
        print(f"Epoch {epoch:4d}: loss={avg_loss:.6f} (form:{avg_form_loss:.6f}, mass:{avg_mass_loss:.4f}), " +
              f"retention={mass_retention:.1%}, mass_w={current_mass_weight:.2f}, -> {tgt}")
        print(f"         sources: [{src_summary}]")

# Save final checkpoint
save_training_checkpoint(
    PHASE2_CHECKPOINT, state.params, current_pool, current_pool_ids,
    epoch, best_loss, best_params, losses, form_losses, mass_losses, transition_history
)

print(f"\nPhase 2 complete. Best loss: {best_loss:.6f}")
if len(times) > 1:
    print(f"Average time per epoch (after JIT): {np.mean(times[1:]):.3f}s")

## Training Curves

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

window = 100

# Total loss
axes[0, 0].plot(losses, alpha=0.3)
if len(losses) > window:
    smoothed = np.convolve(losses, np.ones(window)/window, mode='valid')
    axes[0, 0].plot(range(window-1, len(losses)), smoothed, 'r-', linewidth=2)
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Total Loss')
axes[0, 0].set_title('Total Loss')
axes[0, 0].set_yscale('log')

# Formation loss
axes[0, 1].plot(form_losses, alpha=0.3, color='blue')
if len(form_losses) > window:
    smoothed = np.convolve(form_losses, np.ones(window)/window, mode='valid')
    axes[0, 1].plot(range(window-1, len(form_losses)), smoothed, 'b-', linewidth=2)
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Formation Loss')
axes[0, 1].set_title('Formation Loss (alpha-only)')
axes[0, 1].set_yscale('log')

# Mass conservation
axes[1, 0].plot(mass_losses, alpha=0.3, color='green')
if len(mass_losses) > window:
    smoothed = np.convolve(mass_losses, np.ones(window)/window, mode='valid')
    axes[1, 0].plot(range(window-1, len(mass_losses)), smoothed, 'g-', linewidth=2)
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Mass Conservation Error')
axes[1, 0].set_title('Mass Conservation Loss')

# Transition matrix (includes seed as source)
extended_names = formation_names + ['seed']
num_extended = len(extended_names)
transition_counts = np.zeros((num_extended, num_formations))  # Source can be seed, target is always formation
for src, tgt in transition_history:
    if src < num_extended and tgt < num_formations:
        transition_counts[src, tgt] += 1

im = axes[1, 1].imshow(transition_counts, cmap='Blues', aspect='auto')
axes[1, 1].set_xticks(range(num_formations))
axes[1, 1].set_yticks(range(num_extended))
axes[1, 1].set_xticklabels(formation_names, rotation=45, ha='right')
axes[1, 1].set_yticklabels(extended_names)
axes[1, 1].set_xlabel('Target')
axes[1, 1].set_ylabel('Source')
axes[1, 1].set_title('Transition Counts (includes seed->formation)')
plt.colorbar(im, ax=axes[1, 1])

plt.tight_layout()
plt.show()

## Save Checkpoint

In [ ]:
checkpoint = {
    'params': state.params,
    'config': {
        'grid_size': GRID_SIZE,
        'num_channels': NUM_CHANNELS,
        'hidden_dim': 128,
        'goal_conditioned': True,
        'phase': 2,
        'task': 'transitions',
        'num_formations': num_formations,
        'formation_names': formation_names,
    },
    'metrics': {
        'losses': losses,
        'form_losses': form_losses,
        'mass_losses': mass_losses,
        'best_loss': best_loss,
        'transition_history': transition_history,
    }
}

with open(PHASE2_CHECKPOINT, 'wb') as f:
    pickle.dump(checkpoint, f)

print(f"Checkpoint saved to {PHASE2_CHECKPOINT}")
print(f"Best loss: {best_loss:.6f}")

---

# Evaluation: Formation Transitions

Test that the model can transition between any pair of formations.

In [ ]:
def run_transition(start_state, params, key, target_alpha, num_steps=150):
    """Run a formation transition using target alpha + gradients as spatial guidance.
    
    Args:
        start_state: Starting formation state
        params: Model parameters
        key: PRNG key
        target_alpha: Target formation's alpha channel (H, W) for spatial guidance
        num_steps: Number of NCA steps
    
    Returns:
        List of states (trajectory)
    """
    # Create 4-channel signal with alpha + gradients
    signal = create_formation_signal_spatial(1, GRID_SIZE, GRID_SIZE, target_alpha)
    signal_single = signal[0]
    
    trajectory = [start_state]
    state_curr = start_state
    
    for i in range(num_steps):
        key, subkey = jax.random.split(key)
        # Write first 2 channels to state's parent signal slots
        state_curr = state_curr.at[..., CHILD_CHANNELS.PARENT_SIGNAL_START:CHILD_CHANNELS.PARENT_SIGNAL_END].set(
            signal_single[..., :2]
        )
        # Pass FULL 4-channel signal to model
        state_curr = child_nca.apply(
            {'params': params}, state_curr, subkey, parent_signal=signal_single[..., :MODEL_SIGNAL_CHANNELS]
        )
        trajectory.append(state_curr)
    
    return trajectory

print("Transition function ready (using FULL 4-channel spatial guidance).")
print(f"  Signal channels: {SIGNAL_CHANNELS} (alpha, grad_x, grad_y, reserved)")
print(f"  Model signal channels: {MODEL_SIGNAL_CHANNELS}")

In [ ]:
# Test all pairwise transitions (5x5 = 25 pairs) + seed->formation (5 more)
print("Testing all formation transitions...")

transition_mses = np.zeros((num_formations, num_formations))
transition_retentions = np.zeros((num_formations, num_formations))

# Also test seed -> formation
seed_mses = np.zeros(num_formations)
seed_retentions = np.zeros(num_formations)

key, eval_key = jax.random.split(key)

# Formation -> Formation transitions
for src_idx, src_name in enumerate(formation_names):
    # Create starting state from source formation
    eval_key, state_key = jax.random.split(eval_key)
    start_state = create_formation_state(targets[src_name], state_key)
    initial_mass = float(jnp.sum(start_state[..., 3]))
    
    for tgt_idx, tgt_name in enumerate(formation_names):
        eval_key, run_key = jax.random.split(eval_key)
        
        target = targets[tgt_name]
        
        # Run transition with target alpha as spatial guidance
        trajectory = run_transition(start_state, state.params, run_key, target[..., 3], num_steps=150)
        final = trajectory[-1]
        
        # Compute metrics using ALPHA-ONLY MSE
        mse = float(np.mean((np.array(target[..., 3]) - np.array(final[..., 3]))**2))
        final_mass = float(jnp.sum(final[..., 3]))
        retention = final_mass / initial_mass
        
        transition_mses[src_idx, tgt_idx] = mse
        transition_retentions[src_idx, tgt_idx] = retention

# Seed -> Formation transitions
print("Testing seed -> formation transitions...")
for tgt_idx, tgt_name in enumerate(formation_names):
    eval_key, seed_key, run_key = jax.random.split(eval_key, 3)
    
    # Create Gaussian seed
    seed_state = create_gaussian_seed(GRID_SIZE, GRID_SIZE, NUM_CHANNELS, REFERENCE_MASS, seed_key, sigma=10.0)
    initial_mass = float(jnp.sum(seed_state[..., 3]))
    
    target = targets[tgt_name]
    
    # Run transition
    trajectory = run_transition(seed_state, state.params, run_key, target[..., 3], num_steps=150)
    final = trajectory[-1]
    
    # Compute metrics
    mse = float(np.mean((np.array(target[..., 3]) - np.array(final[..., 3]))**2))
    final_mass = float(jnp.sum(final[..., 3]))
    retention = final_mass / initial_mass
    
    seed_mses[tgt_idx] = mse
    seed_retentions[tgt_idx] = retention

print("Done!")

In [ ]:
# Visualize transition matrices
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# MSE matrix
im1 = axes[0].imshow(transition_mses, cmap='Reds', vmin=0)
axes[0].set_xticks(range(num_formations))
axes[0].set_yticks(range(num_formations))
axes[0].set_xticklabels(formation_names, rotation=45, ha='right')
axes[0].set_yticklabels(formation_names)
axes[0].set_xlabel('Target Formation')
axes[0].set_ylabel('Source Formation')
axes[0].set_title('Transition MSE (lower is better)')
plt.colorbar(im1, ax=axes[0])

# Add text annotations
for i in range(num_formations):
    for j in range(num_formations):
        axes[0].text(j, i, f'{transition_mses[i,j]:.3f}', ha='center', va='center', fontsize=8)

# Retention matrix
im2 = axes[1].imshow(transition_retentions, cmap='Greens', vmin=0.5, vmax=1.5)
axes[1].set_xticks(range(num_formations))
axes[1].set_yticks(range(num_formations))
axes[1].set_xticklabels(formation_names, rotation=45, ha='right')
axes[1].set_yticklabels(formation_names)
axes[1].set_xlabel('Target Formation')
axes[1].set_ylabel('Source Formation')
axes[1].set_title('Mass Retention (1.0 = perfect)')
plt.colorbar(im2, ax=axes[1])

for i in range(num_formations):
    for j in range(num_formations):
        axes[1].text(j, i, f'{transition_retentions[i,j]:.2f}', ha='center', va='center', fontsize=8)

plt.tight_layout()
plt.show()

In [ ]:
# Visualize specific transitions
test_transitions = [
    ('line', 'wedge'),
    ('wedge', 'square'),
    ('square', 'column'),
    ('column', 'phalanx'),
]

fig, axes = plt.subplots(len(test_transitions), 6, figsize=(18, 3*len(test_transitions)))

timesteps = [0, 30, 60, 90, 120, 150]

for row, (src_name, tgt_name) in enumerate(test_transitions):
    src_idx = formation_names.index(src_name)
    tgt_idx = formation_names.index(tgt_name)
    
    eval_key, state_key, run_key = jax.random.split(eval_key, 3)
    start_state = create_formation_state(targets[src_name], state_key)
    target = targets[tgt_name]
    
    # Use target alpha as spatial guidance
    trajectory = run_transition(start_state, state.params, run_key, target[..., 3], num_steps=150)
    
    for col, t in enumerate(timesteps):
        if t < len(trajectory):
            axes[row, col].imshow(trajectory[t][..., 3], cmap='gray', vmin=0, vmax=1)
        axes[row, col].set_title(f't={t}')
        axes[row, col].axis('off')
    
    axes[row, 0].set_ylabel(f'{src_name} -> {tgt_name}', fontsize=10)

plt.suptitle('Formation Transitions Over Time', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Test rotation during transition
print("Testing line -> wedge with different target rotations...")

angles_test = [0, np.pi/4, np.pi/2, np.pi]
angle_labels = ['0°', '45°', '90°', '180°']

fig, axes = plt.subplots(2, len(angles_test), figsize=(3*len(angles_test), 6))

eval_key, state_key = jax.random.split(eval_key)
start_state = create_formation_state(targets['line'], state_key)

for col, (angle, label) in enumerate(zip(angles_test, angle_labels)):
    eval_key, run_key = jax.random.split(eval_key)
    
    # Target wedge at this angle
    rotated_target = rotate_formation(targets['wedge'], float(angle))
    
    # Use rotated target's alpha as spatial guidance
    trajectory = run_transition(start_state, state.params, run_key, rotated_target[..., 3], num_steps=150)
    final = trajectory[-1]
    
    # Target
    axes[0, col].imshow(rotated_target[..., 3], cmap='gray', vmin=0, vmax=1)
    axes[0, col].set_title(f'Target @ {label}')
    axes[0, col].axis('off')
    
    # Result - use alpha-only MSE
    mse = np.mean((np.array(rotated_target[..., 3]) - np.array(final[..., 3]))**2)
    axes[1, col].imshow(final[..., 3], cmap='gray', vmin=0, vmax=1)
    axes[1, col].set_title(f'Result (MSE: {mse:.4f})')
    axes[1, col].axis('off')

plt.suptitle('Line -> Wedge with Rotation', fontsize=14)
plt.tight_layout()
plt.show()

## Success Criteria Check

In [ ]:
print("=" * 60)
print("PHASE 2 SUCCESS CRITERIA: Formation Transitions")
print("=" * 60)
print("(Using alpha-only MSE for shape matching)")

# Analyze transition matrix (formation -> formation)
avg_mse = np.mean(transition_mses)
max_mse = np.max(transition_mses)
avg_retention = np.mean(transition_retentions)
min_retention = np.min(transition_retentions)

# Off-diagonal (actual transitions, not same-to-same)
off_diag_mask = ~np.eye(num_formations, dtype=bool)
avg_transition_mse = np.mean(transition_mses[off_diag_mask])
avg_transition_retention = np.mean(transition_retentions[off_diag_mask])

# Seed -> formation metrics
avg_seed_mse = np.mean(seed_mses)
max_seed_mse = np.max(seed_mses)
avg_seed_retention = np.mean(seed_retentions)
min_seed_retention = np.min(seed_retentions)

print("\n=== Formation -> Formation Metrics (alpha-only) ===")
print(f"  Average MSE (all pairs): {avg_mse:.6f}")
print(f"  Average MSE (transitions only): {avg_transition_mse:.6f}")
print(f"  Worst MSE: {max_mse:.6f}")
print(f"  Average retention: {avg_retention:.1%}")
print(f"  Worst retention: {min_retention:.1%}")

print("\n=== Seed -> Formation Metrics (alpha-only) ===")
print(f"  Average MSE: {avg_seed_mse:.6f}")
print(f"  Worst MSE: {max_seed_mse:.6f}")
print(f"  Average retention: {avg_seed_retention:.1%}")
print(f"  Worst retention: {min_seed_retention:.1%}")

print("\nPer-Transition Analysis (Formation -> Formation):")
for src_idx, src_name in enumerate(formation_names):
    for tgt_idx, tgt_name in enumerate(formation_names):
        if src_idx != tgt_idx:
            mse = transition_mses[src_idx, tgt_idx]
            ret = transition_retentions[src_idx, tgt_idx]
            status = "OK" if mse < 0.02 and ret > 0.85 else "NEEDS WORK"
            print(f"  {src_name:8s} -> {tgt_name:8s}: MSE={mse:.4f}, Retention={ret:.1%} [{status}]")

print("\nPer-Transition Analysis (Seed -> Formation):")
for tgt_idx, tgt_name in enumerate(formation_names):
    mse = seed_mses[tgt_idx]
    ret = seed_retentions[tgt_idx]
    status = "OK" if mse < 0.02 and ret > 0.85 else "NEEDS WORK"
    print(f"  seed     -> {tgt_name:8s}: MSE={mse:.4f}, Retention={ret:.1%} [{status}]")

# Success criteria
print("\n" + "=" * 60)
criteria = [
    ("Avg formation->formation MSE < 0.02", avg_transition_mse, 0.02, avg_transition_mse < 0.02),
    ("Worst formation->formation MSE < 0.05", max_mse, 0.05, max_mse < 0.05),
    ("Avg seed->formation MSE < 0.02", avg_seed_mse, 0.02, avg_seed_mse < 0.02),
    ("Worst seed->formation MSE < 0.05", max_seed_mse, 0.05, max_seed_mse < 0.05),
    ("Avg retention > 85%", min(avg_transition_retention, avg_seed_retention), 0.85, 
     avg_transition_retention > 0.85 and avg_seed_retention > 0.85),
    ("Worst retention > 70%", min(min_retention, min_seed_retention), 0.70, 
     min_retention > 0.70 and min_seed_retention > 0.70),
]

all_passed = True
for name, value, threshold, passed in criteria:
    status = "PASS" if passed else "FAIL"
    print(f"{name}: {value:.4f} (threshold: {threshold}) [{status}]")
    if not passed:
        all_passed = False

print("=" * 60)
if all_passed:
    print("ALL CRITERIA PASSED - Ready for Phase 3!")
    print("\nNext: Implement combat dynamics with adversarial armies")
else:
    print("SOME CRITERIA FAILED - Consider more training")
    print("\nTry: Set RESET=False and run training again")